In [1]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import ast
from sklearn.decomposition import PCA
	
TRAIN = "Train"
EVAL = "Eval"

In [9]:
def plott(embeddings_df, labels_df, title='', task_param='velocity', show_labels=False, ncomponents=3):
    pca = PCA(n_components=ncomponents)
    pca.fit(embeddings_df)
    print('Components:', pca.components_)
    print('Explained variance', pca.explained_variance_)
    print('Explained variance ratio', pca.explained_variance_ratio_)
    embeddings = pca.transform(embeddings_df)
    
    labels = [ast.literal_eval(label.replace("array(", "").replace(", dtype=float32)", "")) for label in labels_df[0].array]
    stages = np.array([
        TRAIN if label['stage'] == 'train' else EVAL 
        for label in labels
    ])
    tasks = np.array([
        float("{0:.2f}".format(
            #label[task_param][0]
            np.linalg.norm(np.array(label[task_param]) - np.array([0, 0.85, 0.175])) if type(label[task_param]) is list else label[task_param]
        ))
        for label in labels
    ])

    if ncomponents == 3:
        fig = px.scatter_3d(embeddings, x=0, y=1, z=2, color=tasks, symbol=stages)
    else:
        fig = px.scatter(embeddings, x=0, y=1, color=tasks, symbol=stages)

    fig.update_layout(
        title=title,
        legend=dict(x=-.1, y=-.1)
    )
    fig.show()

In [23]:
def plott2d(embeddings_df, labels_df, title='', task_param='velocity', show_labels=False, ncomponents=3, task_ax=0, pca_ax=0):
    pca = PCA(n_components=ncomponents)
    pca.fit(embeddings_df)
    print('Components:', pca.components_)
    print('Explained variance', pca.explained_variance_)
    print('Explained variance ratio', pca.explained_variance_ratio_)
    embeddings = pca.transform(embeddings_df)
    
    labels = [ast.literal_eval(label.replace("array(", "").replace(", dtype=float32)", "")) for label in labels_df[0].array]
    stages = np.array([
        TRAIN if label['stage'] == 'train' else EVAL 
        for label in labels
    ])
    tasks = np.array([
        float("{0:.2f}".format(
            label[task_param][task_ax]
            #np.linalg.norm(np.array(label[task_param]) - np.array([0, 0.85, 0.175])) if type(label[task_param]) is list else label[task_param]
        ))
        for label in labels
    ])

    if ncomponents == 3:
        fig = px.scatter_3d(embeddings, x=pca_ax, y=[0]*len(embeddings), z=[0]*len(embeddings), color=tasks, symbol=stages)
    else:
        fig = px.scatter(embeddings, x=0, y=1, color=tasks, symbol=stages)

    fig.update_layout(
        title=title,
        legend=dict(x=-.1, y=-.1)
    )
    fig.show()

In [4]:
experiments = []

for dirname, _, _ in os.walk('output'):
    if 'embeddings' in dirname.split('/')[-1]:
        experiments.append(dirname)

experiments.sort()
for i, experiment in enumerate(experiments):
    print(i, experiment)

0 output/cheetah-vel/2020_02_16_00_49_55/embeddings
1 output/cheetah-vel/2020_02_17_18_09_55/embeddings
2 output/cheetah-vel/2020_03_03_16_20_26/embeddings
3 output/cheetah-vel/2020_03_04_17_23_41/embeddings
4 output/cheetah-vel/2020_03_05_23_41_01/embeddings
5 output/cheetah-vel/2020_03_06_13_23_33/embeddings
6 output/cheetah-vel/2020_03_07_23_12_39/embeddings
7 output/cheetah-vel/2020_03_17_00_33_59/embeddings
8 output/cheetah-vel/2020_03_26_04_30_37/embeddings
9 output/cheetah-vel/2020_03_26_04_34_42/embeddings
10 output/cheetah-vel/2020_03_27_00_51_58/embeddings
11 output/reach-ml1/2020_04_09_01_49_41/embeddings
12 output/reach-ml1/2020_04_15_22_29_42/embeddings
13 output/reach-ml1/2020_04_18_23_23_57/embeddings
14 output/reach-ml1/2020_04_18_23_25_34/embeddings
15 output/reach-ml1/2020_04_21_15_12_49/embeddings
16 output/reach-ml1/2020_04_21_15_13_40/embeddings
17 output/reach-ml1/2020_04_23_11_14_21/embeddings
18 output/reach-ml1/2020_04_23_13_48_31/embeddings
19 output/reach-ml1

In [11]:
results = [
    #(0, 'Int. Obs 1', 'velocity'),
    #(1, 'Int. Ctx 1', 'velocity'),
    #(2, 'PEARL 1', 'velocity'),
    #(3, 'PEARL 2', 'velocity'),
    #(4, 'Int. Ctx 2', 'velocity'),
    #(5, 'Ctx Only Grads', 'velocity'),
    #(6, 'Int. Ctx 3', 'velocity'),
    #(7, 'Int. Ctx 4', 'velocity'),
    #(8, 'L2 0.1', 'velocity'),
    #(9, 'Dropout', 'velocity'),    
    #(10, 'L2 0.01 1', 'velocity', '00002'),
    #(11, 'L2 0.01 2', 'velocity'),
    #(12, 'PEARL 3 (Small)', 'velocity'),
    #(3, 'Reach Normalized', 'goal')
    (14, 'PEARL', 'goal_pos', '00010'),
    #(15, 'PEARL OOD', 'goal_pos', '00010'),
    #(16, 'Scale', 'goal_pos', '00010'),
    #(17, 'L2 0.01', 'goal_pos', '00010'),
    #(18, 'Int Obs OOD', 'goal_pos', '00010'),
    #(19, 'Int Ctx OOD', 'goal_pos', '00010'), # curiosity (ctx) out of dist on reach
    (20, 'Int Ctx', 'goal_pos', '00010'), # curiosty (ctx) on reach
    (21, 'L2 0.01', 'goal_pos', '00010'), # L2 reg on encoder on reach
    #$(22, 'L2 0.01 OOD', 'goal_pos', '00010'), # L2 reg on encoder out of dist on reach
]

In [22]:
for experiment_index, experiment_name, task_param, adapt_index in results:
    path = os.path.join(experiments[experiment_index], adapt_index)
    embeddings_df = pd.read_csv(os.path.join(path, 'default/tensors.tsv'), sep='\t', header=None)
    labels_df = pd.read_csv(os.path.join(path, 'default/metadata.tsv'), sep='\t', header=None)

    plott2d(embeddings_df, labels_df, title=experiment_name, task_param=task_param, show_labels=False)

Components: [[-0.6385614   0.39988923 -0.61985551 -0.18573714  0.11665678]
 [-0.55484331  0.13646082  0.51550978  0.08329862 -0.63311796]
 [ 0.40239439  0.67350249 -0.18885273  0.51231018 -0.29384702]]
Explained variance [0.18350616 0.16158251 0.1445611 ]
Explained variance ratio [0.2810494  0.24747218 0.22140298]


Components: [[-0.42614851  0.86296304 -0.04430536 -0.20969209  0.16660884]
 [-0.0038354  -0.22844197 -0.69402502 -0.66587328  0.15080317]
 [-0.38811206 -0.31926604  0.19456645  0.09844147  0.83659512]]
Explained variance [2.88621306 0.22884214 0.16091988]
Explained variance ratio [0.87121439 0.06907687 0.04857428]


Components: [[ 0.07056456  0.78934036 -0.25159157 -0.45975437  0.31191352]
 [ 0.04902092  0.548908    0.67671624  0.44804845 -0.19391926]
 [ 0.10364949  0.14250915 -0.54132263  0.76356879  0.30476293]]
Explained variance [3.84319357 1.62162978 1.51755527]
Explained variance ratio [0.47950186 0.20232509 0.18934008]
